In [1]:
import numpy as np
import pandas as pd
import dlomix
from dlomix import constants, data, eval, layers, models, pipelines, reports, utils
from dlomix.models import PrositIntensityPredictor
from prosit_t.models import PrositTransformerIntensityPredictor, PrositSimpleIntensityPredictor
from dlomix.losses import masked_spectral_distance, masked_pearson_correlation_distance
from dlomix.constants import ALPHABET_UNMOD

import tensorflow as tf
import wandb
from wandb.keras import WandbCallback
import keras_nlp
print([x for x in dir(dlomix) if not x.startswith("_")])

2023-06-22 15:07:42.975584: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-22 15:07:43.086349: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-22 15:07:43.748527: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-22 15:07:43.748593: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

['META_DATA', 'constants', 'data', 'eval', 'layers', 'losses', 'models', 'pipelines', 'reports', 'utils']


In [2]:
%load_ext autoreload

%autoreload 2

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [4]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [5]:
DEFAULT_CONFIG = {
    "learning_rate": 0.0001,
    "batch_size": 64,
    "embedding_output_dim": 64,
    "ff_dim": 32,
    "num_heads": 16,
    "seq_length": 30,
    "len_fion": 6,
    "vocab_dict": ALPHABET_UNMOD,
    "dropout_rate": 0,
}

In [6]:
project_name = "transforming-prosit"

In [7]:
from prosit_t.data import IntensityDataset

TRAIN_DATAPATH = 'https://raw.githubusercontent.com/wilhelm-lab/dlomix-resources/main/example_datasets/Intensity/proteomeTools_train_val.csv'
BATCH_SIZE = 64
int_data = IntensityDataset(data_source=TRAIN_DATAPATH, seq_length=30,
                            collision_energy_col='collision_energy', batch_size=BATCH_SIZE, val_ratio=0.2, test=False)

2023-06-22 15:07:56.677856: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-22 15:07:57.085667: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43648 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:e2:00.0, compute capability: 8.6


In [8]:
wandb.init(project=project_name, name="replicate_best_model")
cb_wandb = WandbCallback()
callbacks = [cb_wandb]
model = PrositSimpleIntensityPredictor(**DEFAULT_CONFIG)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=DEFAULT_CONFIG["learning_rate"]),
    loss=masked_spectral_distance,
    metrics=[masked_pearson_correlation_distance],
)
model.fit(
    int_data.train_data,
    validation_data=int_data.val_data,
    epochs=60,
    callbacks=callbacks,
)

wandb: Currently logged in as: mamisashvili-lizi. Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/60


2023-06-22 15:08:00.896652: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA A40" frequency: 1740 num_cores: 84 environment { key: "architecture" value: "8.6" } environment { key: "cuda" value: "11020" } environment { key: "cudnn" value: "8100" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 6291456 shared_memory_size_per_multiprocessor: 102400 memory_size: 45768966144 bandwidth: 696096000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


 29/368 [=>............................] - ETA: 1s - loss: 0.7125 - masked_pearson_correlation_distance: 0.6193 

...

INFO:tensorflow:Assets written to: /cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/wandb/run-20230622_150758-k30wxyr5/files/model-best/assets


INFO:tensorflow:Assets written to: /cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/wandb/run-20230622_150758-k30wxyr5/files/model-best/assets
/cmnfs/home/l.mamisashvili/miniconda3/envs/prosit-t/lib/python3.9/site-packages/keras/saving/saved_model/layer_serialization.py:134: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)
wandb: Adding directory to artifact (/cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/wandb/run-20230622_150758-k30wxyr5/files/model-best)... 

OSError: [Errno 122] Disk quota exceeded

wandb: ERROR Error uploading "/cmnfs/home/l.mamisashvili/.local/share/wandb/artifacts/staging/tmpn7l31trz": OSError, [Errno 122] Disk quota exceeded: '/cmnfs/home/l.mamisashvili/.cache/wandb/artifacts/obj/md5/58/tmp_06929f7bf6cbb56a'
wandb: ERROR Uploading artifact file failed. Artifact won't be committed.
wandb: ERROR Error uploading "/cmnfs/home/l.mamisashvili/.local/share/wandb/artifacts/staging/tmp7lnnkp6c": OSError, [Errno 122] Disk quota exceeded: '/cmnfs/home/l.mamisashvili/.cache/wandb/artifacts/obj/md5/69/tmp_8a0c4fd854b20e39'
wandb: ERROR Uploading artifact file failed. Artifact won't be committed.
wandb: ERROR Error uploading "/cmnfs/home/l.mamisashvili/.local/share/wandb/artifacts/staging/tmpm4tmw4mv": FileNotFoundError, [Errno 2] No such file or directory: '/cmnfs/home/l.mamisashvili/.local/share/wandb/artifacts/staging/tmpm4tmw4mv'
wandb: ERROR Uploading artifact file failed. Artifact won't be committed.
--- Logging error ---
Traceback (most recent call last):
  File "/cm

--- Logging error ---
Traceback (most recent call last):
  File "/cmnfs/home/l.mamisashvili/miniconda3/envs/prosit-t/lib/python3.9/logging/__init__.py", line 1087, in emit
    self.flush()
  File "/cmnfs/home/l.mamisashvili/miniconda3/envs/prosit-t/lib/python3.9/logging/__init__.py", line 1067, in flush
    self.stream.flush()
OSError: [Errno 122] Disk quota exceeded
Call stack:
  File "/cmnfs/home/l.mamisashvili/miniconda3/envs/prosit-t/lib/python3.9/threading.py", line 937, in _bootstrap
    self._bootstrap_inner()
  File "/cmnfs/home/l.mamisashvili/miniconda3/envs/prosit-t/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/cmnfs/home/l.mamisashvili/miniconda3/envs/prosit-t/lib/python3.9/site-packages/wandb/sdk/internal/internal_util.py", line 49, in run
    self._run()
  File "/cmnfs/home/l.mamisashvili/miniconda3/envs/prosit-t/lib/python3.9/site-packages/wandb/sdk/internal/internal_util.py", line 100, in _run
    self._process(record)
  File "/cmnfs/